# Lösen der 1-D Diffusionsgleichung (implizit)

In [25]:
using Plots, ExtendableSparse, LinearAlgebra, Printf
using GeoModBox.HeatEquation.OneD

Wenden wir nun das implizite finite Differenzen Schema auf unser 1-D Problem der Wärmediffusionsgleichung an. Zur Wiederholung hier noch einmal das Problem:

## Das Problem

Wir sind an der Änderung der Temperatur *T* mit der Zeit *t* interessiert, für eine bestimmte Anfangstemperatur $T_{0}[x,t_{0}]$ und Randbedingungen; z.B. eine Dikeintrusion in der Lithosphäre (mit *L* = 100 m; *W* = 5 m; *κ* = 10⁻⁶ m²/s).

<img src="../Figures/Exercise02_1.png" alt="drawing" width="350"/> <br>
**Abb. 1.** Sketch des geologischen Problem und des Profils der Anfangstemperaturbedingung. 

Wie lange würde es dauern, bis sich der Dike auf eine bestimmte Temperatur abgekühlt hat?

Zur Bestimmung der Dauer der Abkühlung müssen wir die Diffusionsgleichung der Temperatur lösen (parabolische PDG):

$$ 
\rho c_p \frac{\partial{T}}{\partial{t}} = \frac{\partial{}}{\partial{x}} \left( k\frac{\partial{T}}{\partial{x}} \right), 
$$

wobei *k* die thermische Leitfähigkeit, *T* die Temperatur und $\rho$ die Dichte ist. Durch Umformen (Annahme, dass die thermischen parameter konstant sind!) können wir die Gleichung durch die thermische Diffusivität $\kappa = \frac{k}{\rho c_p}$ ausdrücken: 

$$
\frac{\partial{T}}{\partial{t}} = \kappa \frac{\partial^2{T}}{\partial{x^2}}.
$$

### Finite Differenzen Approximation

Zur numerischen Lösung der Problems mit Hilfe von finiten Differenzen müssen wir zuerst ein numerisches Gitter erstellen (die Diskretisierung):

<img src="../Figures/Exercise02_2.png" alt="drawing" width="600"/> <br>
**Abb. 2.** Numerisches 1-D Gitter für die Diskretisierung der *PDG*. 

Um die Diffusionsgleichung numerisch mit Hilfe des impliziten finiten Differenzen Schema zu lösen müssen wir die *PDG* wie folgt umformulieren:

$$
\frac{T_{i}^{n+1}-T_{i}^{n}}{\Delta t} = \frac{T_{i+1}^{n+1} - 2T_{i}^{n+1} + T_{i-1}^{n+1}}{(\Delta x)^2}
$$

Diese Gleichung können wir umformulieren, so dass wir ein Gleichungssystem erhalten, mit so vielen Gleichungen wie zentralen Gitterpunkten:

$$
-a T_{i-1}^{n+1} + \left(2a + b\right) T_{i}^{n+1} - a T_{i+1}^{n+1} = b T_{i}^{n}, 
$$

mit 
$$
a=\frac{\kappa}{\Delta{x^2}}, b = \frac{1}{\Delta{t}}.
$$

D.h. wir haben ein tridiagonales (drei Diagonalen) Gleichungssystem, welches durch eine Koeffizientenmatrix $\bm{A}$, einem unbekannten Vektor $T^{n+1}$ und einen bekannten Vektor $T^n$ beschrieben werden kann. 

### Parameterdefinitionen

Definieren wir für das Problem erst einmal bestimmte Parameter (physikalische Konstanten, numerische Domaine, Zeitparameters):

In [ ]:
alternative =   3

# Physikalische Parameter ----------------------------------------------- #
L           =   100.0   # Laenge der Modeldomain [m]
Tdike       =   1200.0  # Temperatur des Dikes [C]
Trock       =   300.0   # Temperatur des Umgebungsgesteins [C]
κ           =   1.0e-6  # Thermische Diffusivitaet des Gesteins [m2/s]
W           =   5.0     # Breite des Dykes [m]
# ----------------------------------------------------------------------- #
# Numerische Parameter -------------------------------------------------- #
nc          =   100                 # Anzahl der Gitterpunkte in x-Richtung
Δx          =   L/nc                # Gitterlaenge
xc          =   Δx/2:Δx:(L-Δx/2)   # Gitter
# Iterations
niter       =   10  
ϵ           =   1.0e-10       
# ----------------------------------------------------------------------- #
# Zeit Parameter -------------------------------------------------------- #
day         =   3600.0*24.0     # Sekunden pro Tag
fac         =   1.0 
Δt          =   fac*Δx^2.0 / (2*κ)
tmax        =   365.0*day 
nt          =   ceil(Int,tmax/Δt)
time        =   0
# ----------------------------------------------------------------------- #

### Anfangsbedingungen

Zur Lösung unseres Problem, müssen wir noch die Anfangsbedingungen definieren. Dazu nehmen wir 300 °C für das Umgebungsgestein und 1200 °C für den Dike an. D.h. die Anfangstemperatur ist definiert durch:

$$
T \left(x < \left( \frac{L}{2} - W \right), x > \left( \frac{L}{2} + W \right), t = 0 \right) = 300, 
$$
$$
T \left(x > \left( \frac{L}{2} - W \right), x < \left( \frac{L}{2} + W \right), t = 0 \right) = 1200.
$$

Jetzt wollen wir erst einmal die Anfangsbedingung graphisch darstellen. 

In [ ]:
# Anfangsbedingungen; Temperaturprofil ---------------------------------- #
T       =   (
        T       =    zeros(nc), 
        T0      =    zeros(nc), 
        T_ex    =    zeros(nc+2), 
        R       =    zeros(nc),
        ∂T2∂x2  =    zeros(nc))
T.T    .= Trock                             # Temperatur des Umgebungsgesteins
@. T.T[abs(xc-L/2) <= W/2] =  Tdike       # Temperatur des Dikes
T.T_ex[2:end-1]         .=      T.T
T.T0                    .=      T.T
# ----------------------------------------------------------------------- #
# Plot initial condition ------------------------------------------------ #
p = plot(xc, T.T, xlabel="x [m]", ylabel="T [°C]", 
        title="Temperatur Verteilung nach 
        $(round(time/day, digits=1)) Tagen", 
        ylim=(0, Tdike))
display(p)
# ----------------------------------------------------------------------- #

### Randbedingungen

Da wir zentrale Gitterpunkte für die Temperatur verwenden, liegt kein Gitterpunkt direkt auf den Rändern (hingegen jeder Erwartung ist das allerdings ein Vorteil!). Zur Festlegung der Temperaturrandbedingungen müssen wir uns also der zusätzlichen *Ghost nodes* bediehnen (siehe Abbildung 2), d.h. wir bestimmen die Temperatur auf den *Ghost nodes* um die partielle Differentialgleichung der Temperatur auf dem **nächsten inneren** Gitterpunkt mit Hilfe der finiten Differenzen lösen zu können. Für eine konstante Temperaturbedingung and den Rändern (Dirichlet), können wir die Temperatur der *Ghost nodes* durch lineare Interpolation bestimmem, so dass: 

**West**
$$
T_{Ghost,W} = 2 T_{BC,W} + T_{1},
$$
**East**
$$
T_{Ghost,E} = 2 T_{BC,E} + T_{nc}.
$$

Für konstante Flussrandbedingungen ist die Temperatur an den Rändern gegeben durch: 

**West**
$$
T_{Ghost,W} = T_{1} - c_W \Delta x,
$$
**East**
$$
T_{Ghost,E} = T_{1} + c_E \Delta x,
$$

wobei $c_W = \frac{\partial{T}}{\partial{x}}$ und $c_E = \frac{\partial{T}}{\partial{x}}$ die Flussbedingungen am jeweiligen Rand definieren. 


Da wir ein tridiagonales Gleichungssystem haben, muss man die Koeffizienten und die rechte Seite für die Gleichungen des **ersten** und **letzten inneren Gitterpunktes** in Abhängigkeit der Randbedingungen wie folgt modifizieren (Herleitung siehe Vorlesung): 

#### **Dirichlet**
*West*
$$
\left(3 a + b \right) T_{1}^{n+1} - a T_{2}^{n+1} = b T{1}^{n} + 2 a T_{BC,W}
$$
*East*
$$
- a T_{nc-1}^{n+1} + \left(3 a + b \right) T_{nc}^{n+1} = b T_{nc}^{n} + 2 a T_{BC,E}
$$

#### **Neumann**
*West*
$$
\left(a + b \right) T_{1}^{n+1} - a T_{2}^{n+1} = b T_{1}^n - a c_{W} \Delta{x}
$$
*East*
$$
- a T_{nc-1}^{n+1} + \left(a + b \right) T_{nc}^{n+1}  = b T_{nc}^n - a c_{E} \Delta{x}
$$

Bei der Initialisierung der Randbedingungen bediehnung wir uns eines Tricks und definieren die Temperatur auf den *Ghost nodes* später im Skript: 

In [ ]:
# Randbdingungen -------------------------------------------------------- #
BC   = (
    type = (W=:Dirichlet, E=:Dirichlet),
    # type = (W=:Neumann, E=:Neumann),
    val  = (W=300.0, E=300.0))
#end
# ----------------------------------------------------------------------- #

### Gleichungssystem

Lasst uns nun die Koeffizientenmatrix und rechte Seite initialisieren: 

In [ ]:
# Assemble Coefficient Matrix ------------------------------------------- #
# Definition der Matrix (hier in Sparse Form)
ndof        =   length(T.T)
K           =   ExtendableSparseMatrix(ndof,ndof)
rhs         =   zeros(nc)
# ----------------------------------------------------------------------- #

### Visualisierung

Zur Visualisierung der Ergebnisse als eine Animation in einer GIF Datei, müssen noch der Ort und der Name der Datei festgelegt werden: 

In [ ]:
# Animationssettings ---------------------------------------------------- #
path        =   string("./Results/")
anim        =   Plots.Animation(path, String[] )
filename    =   string("03_1D_implicit_",alternative)
save_fig    =   1
# ----------------------------------------------------------------------- #          

### Zeitschleife

Numerisch können wir nun in einer Zeitschleife die *PDG* auf unterschiedliche Art und Weise lösen (Alternative I ist ausreichend; wer möchte, kann auch die Alternativen II und III lösen). 

In [ ]:
# Timestep loop --------------------------------------------------------- #
for n = 1:nt
    println("Zeitschritt: ",n,", Time: $(round(time/day, digits=1)) [d]")
    if alternative == 1
        a   =   κ / Δx^2
        b   =   1 / Δt

        @. rhs     =   b * T.T

        # Alternative I
        for i = 1:nc  
            # Equation number
            ii          =   i
            # Stencil 
            iW          =   ii - 1
            iC          =   ii
            iE          =   ii + 1   
            # Boundaries 
            inW    =  i==1    ? false  : true
            DirW   = (i==1    && BC.type.W==:Dirichlet) ? 1. : 0.
            NeuW   = (i==1    && BC.type.W==:Neumann  ) ? 1. : 0.
            inE    =  i==nc ? false  : true
            DirE   = (i==nc && BC.type.E==:Dirichlet) ? 1. : 0.
            NeuE   = (i==nc && BC.type.E==:Neumann  ) ? 1. : 0.
            if inE
                K[ii,iE]    = - a
            end
            K[ii,iC]        =   (2 + DirW + DirE - NeuW - NeuE)*a + b
            if inW 
                K[ii,iW]    = - a
            end                            
            # Aenderung der rechten Seite durch die Randbedingungen ------------- #
            rhs[i]  +=  2*a*BC.val.W * DirW - 
                            a*BC.val.W*Δx * NeuW + 
                            2*a*BC.val.E * DirE + 
                            a*BC.val.E*Δx * NeuE
        end            
        T.T     .=   K \ rhs
    elseif alternative == 2            
        BackwardEuler1Dc!(T, κ, Δx, Δt, nc, BC , K, rhs)
    elseif alternative == 3
        for iter = 1:niter
            # Residual iteration
            ComputeResiduals1Dc!(T, κ, Δx, Δt, BC )
            @printf("||R|| = %1.4e\n", norm(T.R)/length(T.R))            
            norm(T.R)/length(T.R) < ϵ ? break : nothing
            # Assemble linear system
            AssembleMatrix1Dc!( κ, Δx, Δt, nc, BC, K )
            # Solve for temperature correction: Cholesky factorisation
            Kc = cholesky(K.cscmatrix)
            # Solve for temperature correction: Back substitutions
            δT = -(Kc\T.R[:])                   
            # Update temperature            
            T.T .= T.T .+ δT            
        end        
        # Überschreiben wir nur das alte Temperaturfeld mit dem neuen      
        @. T.T0     =   T.T  
    end    
    # Berechnung der Zeit ---
    time    =   time + Δt        
    # Plot Lösung ---
    p = plot(xc, T.T, xlabel="x [m]", ylabel="T [°C]", 
            title="Temperatur Verteilung nach 
            $(round(time/day, digits=1)) Tagen", 
            xlim=(0,L),ylim=(0, Tdike))
    if save_fig == 1
        Plots.frame(anim)
    else
        display(p)
    end    
end

Nun müssen wir noch die Animation erstellen und speichern:

In [ ]:
# Speicher Animation ---------------------------------------------------- #
if save_fig == 1
    # Write the frames to a GIF file
    Plots.gif(anim, string( path, filename, ".gif" ), fps = 15)
else
    display(p)
end
foreach(rm, filter(startswith(string(path,"00")), readdir(path,join=true)))
# ----------------------------------------------------------------------- #